In [1]:
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from numpy.linalg import norm
import math
import time

In [2]:
from scipy import sparse

In [3]:
train_data_matrix = sparse.load_npz("train_data_matrix.npz")
train_data_matrix = train_data_matrix.astype(float)

factor_mean_data_matrix = sparse.load_npz("factor_mean_data_matrix.npz").astype(float)

In [4]:
# factor_mean_data_matrix[0].toarray().flatten()

In [5]:
res_dict=np.load('res_dict.npy').item()
user_dict=np.load('user_dict.npy').item()
res_index_to_id = dict((v,k) for k,v in res_dict.items())

In [6]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(train_data_matrix, k = 12)

In [7]:
print(U.shape, sigma.shape, Vt.shape)

(444355, 12) (12,) (12, 28230)


In [8]:
sigma = np.diag(sigma)

In [9]:
print(U.shape, sigma.shape, Vt.shape)

(444355, 12) (12, 12) (12, 28230)


In [10]:
from scipy.sparse import csr_matrix
U = csr_matrix(U)
sigma = csr_matrix(sigma)
Vt = csr_matrix(Vt)

In [11]:
U.dtype

dtype('float64')

In [12]:
print(U.shape, sigma.shape, Vt.shape)

(444355, 12) (12, 12) (12, 28230)


In [13]:
df_test=pd.read_csv('test.csv',header=None, names=['user_id','number_of_res'])

In [14]:
predict=[]
for raw_user,rec_no in df_test.values[0:]:
    if raw_user in user_dict:
         #change raw user id to mapped user id
        user_inp=user_dict[raw_user]
        print(user_inp)
        
        #get un adjuested expected rating
        user_latent_matrix = csr_matrix(U.dot(sigma))[user_inp]
        user_predicted_ratings = user_latent_matrix[0].dot(Vt).toarray()
#         print(user_predicted_ratings.toarray())
#         print(factor_mean_data_matrix[user_inp].toarray())
        user_predicted_ratings += factor_mean_data_matrix[user_inp].toarray()
        top_res = sorted(list(enumerate(user_predicted_ratings[0])), key=lambda x:x[1], reverse = True)
#         print(top_res)
        output = []
        for i in range(rec_no):
            output.append(res_index_to_id[top_res[i][0]])
        predict.append(output)
    else:
        predict.append([0]*rec_no)

1
2
3
4
8
9
10
13
16
17
24
32
38
42
43
47
48
51
52
54
64
65
70
73
75
83
86
89
91
94
98
99
101
107
109
111
115
116
131
132
138
141
142
143
147
149
150
151
154
163
164
167
171
175
178
181
184
186
188
193
200
205
211
218
219
222
228
234
235
239
244
252
260
264
270
273
274
278
280
281
286
287
289
298
302
303
306
308
309
312
315
316
317
318
320
322
329
334
335
342
343
347
350
351
353
360
367
383
387
388
389
391
395
404
406
408
412
416
417
420
428
431
432
440
441
443
444
445
447
448
449
451
453
455
460
461
467
469
470
476
479
480
490
491
492
493
495
497
498
500
502
508
509
510
518
523
524
525
526
529
540
541
575
580
581
582
583
584
587
598
599
602
609
610
611
612
616
617
619
621
623
624
626
629
630
635
636
637
641
643
648
649
651
652
656
669
673
676
679
680
681
684
694
696
698
702
724
728
736
738
739
744
745
765
772
781
786
795
803
804
805
806
807
816
824
827
837
845
859
861
868
869
880
881
894
903
906
922
930
937
938
943
945
949
951
957
959
967
972
973
978
987
993
998
999
1006
1010
1011
101

7689
7690
7692
7695
7699
7703
7705
7709
7710
7720
7742
7745
7752
7755
7762
7764
7768
7771
7781
7782
7787
7793
7803
7804
7812
7825
7832
7833
7838
7843
7845
7848
7849
7851
7852
7855
7858
7863
7866
7873
7876
7880
7881
7883
7884
7887
7888
7890
7891
7893
7896
7898
7901
7917
7920
7921
7923
7934
7938
7939
7944
7946
7949
7950
7951
7955
7959
7961
7963
7964
7965
7972
7975
7997
7999
8004
8010
8011
8017
8018
8021
8038
8044
8046
8052
8058
8060
8067
8069
8076
8077
8079
8086
8093
8098
8099
8104
8114
8115
8119
8121
8128
8130
8137
8138
8141
8149
8152
8155
8157
8160
8162
8164
8171
8172
8180
8186
8187
8198
8205
8206
8225
8246
8252
8254
8270
8272
8273
8274
8280
8286
8294
8298
8303
8305
8310
8311
8313
8314
8317
8318
8319
8323
8333
8335
8343
8344
8345
8349
8356
8369
8370
8379
8381
8389
8394
8403
8405
8410
8417
8429
8432
8441
8448
8463
8473
8494
8498
8515
8516
8517
8520
8523
8524
8528
8532
8533
8536
8544
8548
8559
8560
8562
8563
8565
8570
8574
8584
8587
8600
8607
8617
8620
8623
8632
8633
8635
8636
8637
8639


14789
14796
14798
14800
14806
14809
14810
14813
14820
14825
14827
14828
14831
14832
14834
14836
14839
14843
14844
14859
14861
14874
14893
14897
14901
14917
14922
14924
14925
14932
14933
14934
14937
14942
14948
14952
14958
14960
14964
14970
14982
14994
14995
14996
15006
15008
15017
15020
15029
15036
15038
15058
15061
15065
15066
15076
15077
15084
15088
15095
15110
15118
15129
15146
15159
15164
15181
15183
15188
15210
15219
15220
15233
15246
15253
15255
15259
15260
15268
15273
15276
15296
15298
15300
15303
15316
15320
15323
15331
15334
15336
15343
15348
15352
15353
15360
15361
15370
15374
15388
15393
15402
15405
15407
15411
15416
15417
15424
15427
15451
15452
15455
15457
15459
15463
15470
15474
15475
15483
15490
15507
15508
15513
15516
15523
15524
15531
15533
15544
15546
15550
15553
15555
15556
15560
15563
15568
15578
15581
15582
15595
15600
15612
15613
15614
15630
15633
15638
15641
15642
15643
15645
15657
15665
15673
15675
15676
15678
15686
15687
15697
15701
15702
15705
15710
15729
1573

21841
21850
21853
21870
21871
21876
21878
21879
21887
21900
21901
21905
21906
21907
21916
21917
21921
21922
21925
21926
21927
21929
21937
21944
21973
21989
21991
22001
22007
22014
22016
22023
22026
22031
22043
22047
22049
22061
22068
22071
22084
22095
22096
22098
22101
22104
22105
22108
22111
22112
22113
22116
22120
22121
22132
22134
22141
22144
22149
22150
22157
22158
22166
22169
22170
22186
22194
22197
22208
22211
22218
22219
22225
22227
22232
22250
22252
22256
22262
22263
22265
22276
22284
22287
22289
22290
22294
22307
22308
22309
22310
22311
22319
22322
22324
22327
22328
22331
22335
22346
22351
22357
22360
22362
22364
22372
22376
22380
22381
22387
22394
22405
22412
22417
22418
22420
22429
22431
22440
22447
22448
22450
22452
22453
22454
22463
22465
22469
22474
22480
22482
22483
22488
22489
22490
22491
22498
22507
22518
22519
22522
22527
22542
22545
22549
22555
22560
22563
22564
22577
22581
22583
22587
22596
22598
22605
22606
22609
22612
22614
22630
22635
22639
22650
22656
22660
2266

29156
29158
29160
29164
29165
29175
29188
29193
29203
29206
29216
29223
29225
29230
29231
29237
29245
29248
29258
29263
29266
29271
29280
29285
29287
29291
29297
29299
29308
29312
29314
29317
29321
29328
29332
29337
29344
29345
29357
29358
29366
29367
29368
29369
29371
29374
29375
29376
29379
29380
29382
29397
29402
29407
29408
29418
29419
29424
29427
29447
29449
29455
29456
29462
29463
29466
29473
29481
29483
29487
29488
29492
29503
29505
29519
29543
29549
29551
29554
29555
29569
29570
29572
29573
29576
29577
29580
29582
29595
29598
29599
29611
29612
29613
29615
29621
29623
29632
29645
29657
29661
29662
29665
29669
29677
29682
29697
29700
29701
29703
29706
29720
29727
29734
29744
29750
29754
29756
29767
29769
29775
29793
29796
29798
29804
29806
29813
29815
29819
29836
29853
29858
29860
29868
29879
29884
29904
29906
29908
29909
29912
29913
29914
29917
29918
29922
29931
29932
29933
29935
29936
29941
29943
29944
29972
29973
29974
29986
29997
30000
30001
30002
30003
30013
30020
30027
3003

...

52851
52855
52866
52887
52888
52889
52896
52907
52910
52914
52919
52932
52941
52943
52947
52948
52950
52952
52953
52955
52957
52959
52968
52973
52975
52977
52980
52983
52993
52997
52999
53006
53012
53018
53020
53023
53025
53030
53033
53042
53043
53046
53051
53058
53061
53064
53071
53073
53091
53101
53108
53109
53119
53121
53123
53130
53131
53133
53144
53149
53157
53160
53161
53164
53166
53170
53172
53190
53191
53195
53196
53218
53221
53223
53226
53232
53233
53235
53240
53244
53248
53252
53254
53258
53262
53263
53265
53274
53280
53282
53286
53288
53290
53291
53293
53294
53296
53300
53301
53308
53310
53311
53312
53314
53315
53316
53333
53336
53338
53341
53348
53370
53376
53378
53380
53383
53386
53393
53404
53406
53407
53408
53414
53418
53422
53426
53430
53439
53441
53458
53459
53460
53473
53477
53484
53490
53492
53501
53502
53507
53508
53510
53528
53530
53531
53535
53543
53547
53555
53561
53564
53568
53572
53573
53593
53596
53600
53606
53612
53618
53635
53639
53640
53641
53647
53652
5365

61444
61453
61469
61470
61483
61485
61511
61519
61523
61526
61533
61542
61544
61546
61548
61549
61554
61558
61562
61568
61571
61574
61586
61587
61592
61597
61603
61612
61616
61624
61625
61627
61644
61653
61657
61660
61682
61688
61701
61712
61721
61722
61732
61734
61740
61746
61750
61751
61753
61757
61761
61785
61806
61808
61809
61814
61817
61829
61833
61857
61863
61868
61878
61881
61886
61887
61897
61910
61924
61935
61941
61949
61959
61961
61962
61970
61976
61977
61979
61980
61988
61993
61999
62003
62009
62011
62013
62015
62018
62026
62036
62049
62051
62058
62072
62077
62079
62085
62088
62103
62104
62106
62113
62121
62128
62131
62132
62136
62142
62144
62148
62162
62164
62167
62170
62178
62186
62187
62204
62220
62223
62224
62243
62245
62248
62284
62288
62295
62296
62297
62303
62305
62306
62307
62311
62317
62319
62331
62336
62341
62345
62348
62349
62357
62358
62363
62376
62389
62393
62412
62416
62418
62425
62426
62427
62430
62431
62455
62456
62465
62473
62480
62481
62492
62495
62499
6250

69763
69767
69779
69780
69791
69792
69793
69797
69806
69809
69812
69813
69821
69824
69825
69829
69831
69834
69855
69857
69860
69862
69866
69868
69877
69886
69889
69891
69895
69896
69909
69934
69947
69948
69952
69954
69955
69968
69969
69982
69986
69987
69990
70001
70008
70010
70021
70028
70035
70037
70045
70055
70061
70076
70083
70084
70088
70090
70096
70101
70104
70108
70111
70112
70122
70123
70127
70131
70137
70140
70146
70150
70151
70162
70163
70175
70182
70186
70189
70200
70211
70214
70217
70221
70231
70239
70245
70254
70256
70259
70266
70271
70273
70277
70280
70281
70287
70291
70296
70298
70300
70301
70303
70307
70312
70315
70321
70323
70335
70344
70352
70353
70357
70360
70374
70377
70379
70380
70386
70389
70397
70428
70433
70437
70444
70459
70461
70463
70465
70470
70472
70486
70495
70503
70523
70526
70540
70559
70593
70606
70608
70612
70613
70614
70617
70627
70630
70636
70638
70662
70663
70664
70666
70667
70670
70682
70693
70695
70697
70699
70708
70710
70716
70717
70726
70735
7073

77467
77469
77475
77477
77483
77488
77489
77493
77500
77501
77503
77510
77518
77523
77524
77525
77533
77534
77542
77543
77545
77557
77562
77564
77565
77568
77569
77576
77580
77583
77585
77591
77604
77607
77618
77619
77620
77630
77646
77655
77661
77667
77668
77670
77676
77678
77686
77689
77704
77707
77717
77728
77730
77735
77744
77757
77759
77762
77766
77768
77774
77790
77792
77797
77800
77806
77811
77821
77822
77826
77830
77832
77834
77836
77838
77840
77843
77853
77856
77874
77881
77885
77888
77889
77895
77901
77902
77916
77918
77923
77938
77944
77962
77964
77972
77975
77985
77989
77990
77991
78000
78001
78013
78016
78019
78020
78026
78028
78041
78044
78048
78062
78079
78084
78094
78098
78100
78101
78105
78107
78123
78125
78131
78135
78140
78148
78152
78157
78166
78177
78178
78193
78195
78196
78203
78207
78209
78211
78217
78220
78234
78235
78239
78248
78250
78255
78260
78265
78271
78274
78280
78282
78283
78288
78291
78292
78299
78301
78304
78313
78314
78319
78320
78322
78340
78346
7835

85006
85007
85032
85037
85042
85066
85071
85109
85110
85115
85116
85117
85122
85143
85153
85157
85158
85159
85161
85163
85169
85178
85186
85191
85193
85199
85210
85229
85244
85246
85248
85261
85265
85270
85276
85280
85281
85286
85295
85306
85324
85326
85329
85340
85346
85353
85363
85373
85379
85386
85407
85410
85411
85412
85415
85419
85421
85422
85426
85432
85434
85437
85444
85445
85453
85458
85463
85464
85470
85473
85474
85487
85494
85506
85508
85514
85518
85530
85548
85551
85553
85570
85571
85573
85576
85589
85590
85592
85596
85597
85601
85607
85612
85617
85636
85640
85668
85677
85696
85710
85714
85716
85729
85732
85733
85734
85739
85756
85764
85783
85789
85790
85802
85820
85827
85848
85863
85864
85866
85869
85883
85885
85899
85915
85926
85931
85944
85951
85966
85975
85977
85985
85986
85987
85988
85989


In [15]:
predict

[[4,
  12340,
  2851,
  7699,
  203990,
  2785,
  149852,
  138064,
  3488,
  5720,
  122938,
  2496,
  8299,
  8250,
  733,
  13581,
  98,
  63117,
  12272,
  14266],
 [12340,
  2785,
  2851,
  203990,
  224,
  119931,
  5290,
  7648,
  7644,
  184232,
  5720,
  3609,
  184778,
  18580,
  17612,
  14527,
  227018,
  232909,
  139616,
  18913,
  9233,
  22458,
  8964,
  131908,
  3137,
  121339,
  128986,
  2386,
  15662,
  361],
 [7699,
  8344,
  2851,
  10105,
  8201,
  10183,
  4285,
  14062,
  131724,
  2559,
  9053,
  1262,
  12404,
  8373,
  12340,
  11596,
  120321,
  166774,
  172206,
  117954,
  149852,
  7661,
  869,
  8742,
  27080,
  2858,
  1714,
  22060,
  1486,
  7198,
  9934,
  203990,
  9023,
  14,
  125319,
  2097,
  218426,
  7399,
  8476,
  184232,
  130419,
  138064,
  11553,
  130254,
  9045,
  120304,
  5720,
  122938,
  22863,
  131997,
  13581,
  8564,
  63117,
  7648,
  8250,
  3959,
  2785,
  150896,
  9458,
  7644,
  14149,
  2496,
  10191,
  8669,
  8271,
 

In [16]:
predict[0]

[4,
 12340,
 2851,
 7699,
 203990,
 2785,
 149852,
 138064,
 3488,
 5720,
 122938,
 2496,
 8299,
 8250,
 733,
 13581,
 98,
 63117,
 12272,
 14266]

In [17]:
import csv
with open("predict" + str(batch) + ".txt","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(predict)

In [18]:
# user_latent_matrix[0].dtype

In [19]:
# print(user_latent_matrix.shape, Vt.shape)

In [20]:
# user_predicted_ratings = user_latent_matrix[0].dot(Vt) #+ user_ratings_mean.reshape(-1, 1)

In [21]:
# sorted(list(enumerate(user_predicted_ratings.toarray()[0])), key=lambda x:x[1], reverse = True)

In [22]:
svd = set(predict[0])

old = set([109,879,1486,1714,1930,2977,3509,4309,5955,6601,7124,7668,8250,9010,9045,9149,9864,9948,10656,12665,15802,16437,17256,17510,18319,20574,22640,24851,26244,44759,74797,112706,113716,118543,119780,125428,128204,138091,152026,152842,153577,165953,166733,175697,201275,209822,215047,227018,241964,278366,2,88,161,162,245,369,869,940,1286,1870,2243,2244,2288,2294,2331,2386,2496,2695,2851,2973,3054,3057,3160,3258,3307,3406,3609,3815,3968,4021,4240,4335,4883,5269,5345,5771,7678,7744,7824,7828,7835,8228,8297,8398,8766,9053,9273,9442,9545,9551,9658,9745,9752,9934,10126,10161,10445,10505,10567,10668,10715,10783,10937,11203,11251,11308,11361,11466,11559,11624,11730,11870,11894,11910,11974,12178,12208,12314,12358,12406,12737,12955,13135,13444,13495,13585,13593,13651,13866,13905,14055,14263,14719,14906,14909,15215,15264,15423,15875,16043,16125,16160,17451,17625,18696,18946,19102,19310,20909,22099,22208,22738,23084,23136,23883,25156,25725,27165,27607,27773,58712,60135,61558,61980,62803,62832,66199,67761,67912,70843,71925,72370,73204,76874,86428,87796,119269,119406,120304,123040,124836,125319,125891,127661,129913,130356,130597,131006,135366,135641,140665,144015,144945,145645,148136,148516,151661,152406,154007,155722,155805,157378,158246,158958,159446,160535,160606,162245,163990,165359,166346,171306,173337,175563,175768,175798,176121,176633,181306,181315,183460,186561,187403,188584,189208,189964,191689,191746,192423,193011,195012,196551,197302,197814,198823,199029,201112,203861,204185,204262,207272,212399,214482,215711,216256,218405,219386,226680,226958,229918,232738,234400,237377,238008,241487,244001,247568])

old2 = set([2949,13135,203990,221456,249644,270936,1537,2851,7338,17898,23925,24120,28129,119931,122253,152842,154878,183491,184232,184778,198823,212520,222629,225723,232738,251343,255144,255598,4,361])

In [23]:
len(svd)

20

In [24]:
len(svd - old2)

17